<a href="https://colab.research.google.com/github/Azizkhaled/NLP-with-Aziz/blob/main/NER_On_Sub_reddits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Pulling Data from Reddit

In [2]:
!pip install praw

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 3.7 MB/s eta 0:00:00


In [9]:
import praw
import pandas as pd

In [5]:
CLIENT_ID = <Use your own :)>
SECRET_TOKEN = <Use your own :)>

reddit = praw.Reddit(client_id=CLIENT_ID,
                     client_secret=SECRET_TOKEN,
                     user_agent='Intelligent_Drop_999')

# get 10 hot posts from the MachineLearning subreddit

SUB = 'investing'
# hot_posts = reddit.subreddit('investing').hot(limit=20)

# for post in hot_posts:
#     print(post.title)

In [12]:
# initialize dataframe to store data
df = []

ml_subreddit = reddit.subreddit(SUB)

#pulling the 100 hottest posts
for post in ml_subreddit.hot():
    df.append([post.id, post.created,post.subreddit, post.title, post.selftext, post.upvote_ratio, post.ups, post.downs, post.score])


df = pd.DataFrame(df, columns=['id', 'created', 'subreddit',  'title', 'selftext', 'upvote_ratio', 'ups', 'downs', 'score'])


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



In [11]:
df.tail()

,id,created,subreddit,title,selftext,upvote_ratio,ups,downs,score
95,15mqzsv,1.691614e+09,investing,The reason why the stock market ALWAYS is over...,I know that to figure out whether the broad ma...,0.67,46,0,46
96,15ngovr,1.691684e+09,investing,"ROA, ROE & ROI - How to use and across industries","All,\n\nI guess 3 metrics don't really get a l...",0.67,1,0,1
97,15nfjkg,1.691681e+09,investing,European broker to trade US options,I have been looking for a trading platform tha...,0.40,0,0,0
98,15n6zsn,1.691658e+09,investing,Daily General Discussion and Advice Thread - A...,Have a general question? Want to offer some c...,0.67,2,0,2
99,15n621q,1.691655e+09,investing,Money Market Sweep Accounts,Recently learned that some brokerage accounts ...,0.40,0,0,0


In [13]:
#Saving the pulled data
df = df.replace({'|': ''}, regex=True)
SUB = 'investing'
df.to_csv(f'reddit_{SUB}.csv', sep='|', index=False)



# Extract ORGs from data

## Intitilize model

In [19]:
import spacy
import pandas as pd

nlp = spacy.load('en_core_web_sm')

## Read data

In [20]:
df = pd.read_csv('./reddit_investing.csv', sep='|')
df.head()

,id,created,subreddit,title,selftext,upvote_ratio,ups,downs,score
0,15qpamr,1.692004e+09,investing,Daily General Discussion and Advice Thread - A...,Have a general question? Want to offer some c...,0.78,5,0,5
1,15nalit,1.691669e+09,investing,"August 10, 2023 United States CPI Release Disc...","Please limit all discussions of the US July, 2...",0.79,22,0,22
2,15r7wh1,1.692049e+09,investing,Treasuries at 5% plus now,I just pulled my savings from Apple Savings an...,0.93,102,0,102
3,15rdx4p,1.692063e+09,investing,"After watching The Big Short, I have a question","Two questions. The first, if all the banks wer...",0.86,50,0,50
4,15r1a4n,1.692034e+09,investing,Can we back off our retirement accounts?,Wife (28) and I (31) have been heavily contrib...,0.74,89,0,89


## Function to use extract Entity identifies as organizations (ORGs)

In [21]:
def get_orgs(text):
    # process the text with our SpaCy model to get named entities
    doc = nlp(text)
    # initialize list to store identified organizations
    org_list = []
    # loop through the identified entities and append ORG entities to org_list
    for entity in doc.ents:
        if entity.label_ == 'ORG':
            org_list.append(entity.text)
    # if organization is identified more than once it will appear multiple times in list
    # we use set() to remove duplicates then convert back to list
    org_list = list(set(org_list))
    return org_list

## add column for the organizations mention in each text

In [23]:
df['organizations'] = df['selftext'].apply(get_orgs)
df.head()

,id,created,subreddit,title,selftext,upvote_ratio,ups,downs,score,organizations
0,15qpamr,1.692004e+09,investing,Daily General Discussion and Advice Thread - A...,Have a general question? Want to offer some c...,0.78,5,0,5,"[wiki, FAQ]"
1,15nalit,1.691669e+09,investing,"August 10, 2023 United States CPI Release Disc...","Please limit all discussions of the US July, 2...",0.79,22,0,22,"[the Bureau of Labor Statistics, U.S. Bureau o..."
2,15r7wh1,1.692049e+09,investing,Treasuries at 5% plus now,I just pulled my savings from Apple Savings an...,0.93,102,0,102,"[VOO, US Treasury, Apple Savings]"
3,15rdx4p,1.692063e+09,investing,"After watching The Big Short, I have a question","Two questions. The first, if all the banks wer...",0.86,50,0,50,[FDIC]
4,15r1a4n,1.692034e+09,investing,Can we back off our retirement accounts?,Wife (28) and I (31) have been heavily contrib...,0.74,89,0,89,"[Air BnB, HSA]"


## Function to get the most mentioned ORG

In [26]:
from collections import Counter

In [27]:
# merge organizations column into one big list
orgs = df['organizations'].to_list()
orgs = [org for sublist in orgs for org in sublist]
orgs[:10]

['wiki',
 'FAQ',
 'the Bureau of Labor Statistics',
 'U.S. Bureau of Labor Statistics',
 'BLS',
 'VOO',
 'US Treasury',
 'Apple Savings',
 'FDIC',
 'Air BnB']

In [28]:
# create dictionary of organization mention frequency
org_freq = Counter(orgs)

In [29]:
org_freq.most_common(10)

[('wiki', 5),
 ('FAQ', 5),
 ('VOO', 5),
 ('Fidelity', 5),
 ('Apple', 3),
 ('QQQ', 2),
 ('Google', 2),
 ('IPO', 2),
 ('Meta', 2),
 ('Robinhood', 2)]